In [1]:
import pandas as pd
import datetime
import numpy as np
import statistics
import yfinance as yf

In [2]:
df = pd.read_excel('/Users/brunopimentel/Downloads/Historic_final_Bruno.xlsx')
df['time_setup'] = df['time_setup'].dt.date

In [3]:
capital = 100000

df2 = pd.DataFrame(df.groupby(['time_setup']).sum())
idx = pd.date_range('2021-09-17', '2021-10-06')
df2.index = pd.DatetimeIndex(df2.index)
df2 = df2.reindex(idx, fill_value=0)

data =pd.DataFrame()
data['time'] = df2.index.unique()
data['profit_d'] = np.array(df2['profit'])
data['profit_acm_d'] = data['profit_d'] + capital
i = 0
n = len(data)-1
for i in range (n):
    data['profit_acm_d'][i+1] = data['profit_d'][i+1] + data['profit_acm_d'][i]

<ipython-input-3-ceb950e36ece>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['profit_acm_d'][i+1] = data['profit_d'][i+1] + data['profit_acm_d'][i]


In [4]:
def f_evolucion_capital(data):
    capital = 100000

    df2 = pd.DataFrame(df.groupby(['time_setup']).sum())
    idx = pd.date_range('2021-09-17', '2021-10-06')
    df2.index = pd.DatetimeIndex(df2.index)
    df2 = df2.reindex(idx, fill_value=0)

    data =pd.DataFrame()
    data['time'] = df2.index.unique()
    data['profit_d'] = np.array(df2['profit'])
    data['profit_acm_d'] = data['profit_d'] + capital
    i = 0
    n = len(data)-1
    for i in range (n):
        data['profit_acm_d'][i+1] = data['profit_d'][i+1] + data['profit_acm_d'][i]
    return data
# Llenar datos entre fechas 

In [5]:
f_evolucion_capital(df)

<ipython-input-4-e78c21a7386e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['profit_acm_d'][i+1] = data['profit_d'][i+1] + data['profit_acm_d'][i]


,time,profit_d,profit_acm_d
0,2021-09-17,-0.68,99999.32
1,2021-09-18,0.00,99999.32
2,2021-09-19,0.00,99999.32
3,2021-09-20,0.00,99999.32
4,2021-09-21,-812.60,99186.72
5,2021-09-22,4.24,99190.96
6,2021-09-23,946.53,100137.49
7,2021-09-24,0.00,100137.49
8,2021-09-25,0.00,100137.49
9,2021-09-26,0.00,100137.49


In [6]:
# Sharpe Ratio Original:
rend = np.log(data.profit_acm_d) - np.log(data.profit_acm_d.shift(1))
rend = rend.dropna()

prom_rend_log = np.mean(rend)

rf = .05

des_est = statistics.pstdev(rend)

Sharpe_O = (prom_rend_log-rf)/des_est

In [7]:
# Descarga de precios standard and poor's 500
fecha_inicial = df['time_setup'][0]
fecha_final = df['time_setup'][len(df)-1]

precios_sp500 = yf.download('^GSPC', start= fecha_inicial, end=fecha_final, interval='1d') 
precios_sp500.drop(['Open', 'High', 'Low', 'Volume', 'Adj Close'], axis=1, inplace=True)
precios_sp500['Date'] = precios_sp500.index

[*********************100%***********************]  1 of 1 completed


In [15]:
# Sharpe Ratio Actualizado:
rend_porta = np.log(data.profit_acm_d) - np.log(data.profit_acm_d.shift(1))
rend_sp500 = np.log(precios_sp500.Close) - np.log(precios_sp500.Close.shift(1))

rend_porta = rend_porta.dropna()
rend_sp500 = rend_sp500.dropna()

prom_rend_porta_log = np.mean(rend_porta)
prom_rend_sp500_log = np.mean(rend_sp500)

des_est = prom_rend_porta_log - prom_rend_sp500_log

Sharpe_A = (prom_rend_porta_log - prom_rend_sp500_log)/des_est

In [46]:
# DrawDown (Capital):
data  = f_evolucion_capital(df)
data_min = data['profit_d'].min()
data_max = data[0:posicion_min+1]['profit_acm_d'].max()

posicion_max = data[0:posicion_min+1]['profit_acm_d'].idxmax()
posicion_min = data['profit_d'].idxmin()


fecha_inicial_dd = data['time'][posicion_max]
fecha_final_dd = data['time'][posicion_min]

<ipython-input-4-e78c21a7386e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['profit_acm_d'][i+1] = data['profit_d'][i+1] + data['profit_acm_d'][i]


In [66]:
# Drown up de los -812.60 a los 1009.75
data  = f_evolucion_capital(df)
data_min_du = data['profit_d'].min()
data_max_du = data['profit_d'].max()

posicion_max = data['profit_d'].idxmax()
posicion_min = data['profit_d'].idxmin()


fecha_inicial_du = data['time'][posicion_min]
fecha_final_du = data['time'][posicion_max]

<ipython-input-4-e78c21a7386e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['profit_acm_d'][i+1] = data['profit_d'][i+1] + data['profit_acm_d'][i]


In [68]:
estadisticas = pd.DataFrame()
estadisticas['metrica'] = ['sharpe_original', 'sharpe_actualizado', 'drawdown_capi', 'drawdown_capi', 
                           'drawdown_capi', 'drawup_capi', 'drawup_capi', 'drawup_capi']
estadisticas[''] = ['Cantidad', 'Cantidad', 'Fecha Inicial', 'Fecha Final', 'DrawDown $ (capital)',
                    'Fecha Inicial', 'Fecha Final', 'DrawDown $ (capital)']
estadisticas['Valor'] = [Sharpe_O, Sharpe_A, fecha_inicial_dd, fecha_final_dd, data_min, fecha_inicial_du,
                         fecha_final_du, data_max_du]
estadisticas

,metrica,,Valor
0,sharpe_original,Cantidad,-12.112049
1,sharpe_actualizado,Cantidad,1.0
2,drawdown_capi,Fecha Inicial,2021-10-06 00:00:00
3,drawdown_capi,Fecha Final,2021-09-21 00:00:00
4,drawdown_capi,DrawDown $ (capital),-812.6
5,drawup_capi,Fecha Inicial,2021-09-21 00:00:00
6,drawup_capi,Fecha Final,2021-10-06 00:00:00
7,drawup_capi,DrawDown $ (capital),1009.75
